In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import string
import os
from PIL import Image
from time import time
from keras.layers import Dense, Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.utils import img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras
from keras.models import Model
from keras import Input, layers
from tqdm import tqdm

In [3]:
object_model=load_model('/content/drive/MyDrive/5조/모델/07.21 객체검출 모델/best_model.h5')
scenery_model=load_model('/content/drive/MyDrive/5조/모델/07.21 배경검출 모델/best_model.h5')

In [ ]:
# 이미지의 벡터화를 진행하는 코드

# Load the inception v3 model
model_v3 = InceptionV3(weights='imagenet')

# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model_v3.input, model_v3.layers[-2].output)


96124928/96112376 [==============================] - 1s 0us/step


In [ ]:
# Function to encode a given image into a vector of size (2048, )
def encode(image_path):
    img = keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    # Convert image to numpy array of 3-dimensions
    x = img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    fea_vec = model_new.predict(x) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec   # 이 데이터가 input data로 사용됨

In [ ]:
image_path='/content/temp1/'    # 분석할 이미지가 있는 디렉토리 지정 필요
data=[]
for image in tqdm(  ):
  path=image_path+image     
  data.append( np.array(encode(path)) )

data=np.array(data)

100%|██████████| 3048/3048 [14:48<00:00,  3.43it/s]


array([[0.5589647 , 0.2209115 , 0.10350508, ..., 0.61137414, 0.29394227,
        0.0053378 ],
       [0.15149772, 1.1019528 , 0.66359836, ..., 0.38602915, 0.16809331,
        0.470982  ],
       [0.34664848, 0.2418502 , 1.325381  , ..., 0.09563419, 0.12009779,
        0.09665566],
       ...,
       [0.23977375, 0.18751779, 1.8036178 , ..., 0.9742502 , 0.6386868 ,
        1.3077613 ],
       [0.05636256, 0.36781067, 0.5337228 , ..., 0.5544225 , 0.7525874 ,
        0.        ],
       [0.16076939, 0.257062  , 0.01088384, ..., 0.06123613, 0.440277  ,
        0.18932901]], dtype=float32)

In [ ]:
# 객체검출 결과
y_object_pred=[]
label=object_model.predict(data.reshape(-1,2048))
for sample in label:
  y_object_pred.append([1 if i>=0.5 else 0 for i in sample ] )
y_object_pred = np.array(y_object_pred)

In [ ]:
# 풍경검출 결과
y_scenery_pred=[]
label=scenery_model.predict(data.reshape(-1,2048)))
for sample in label:
  y_scenery_pred.append([1 if i>=0.5 else 0 for i in sample ] )
y_scenery_pred = np.array(y_scenery_pred)